In [18]:
from rich import pretty
from rich import traceback
import numpy as np


pretty.install(indent_guides=True, expand_all=True)
traceback.install(show_locals=True, indent_guides=True)

%load_ext rich

The rich extension is already loaded. To reload it, use:
  %reload_ext rich


# Classic Bits

## 1. Operator

### 1.1 And

In [11]:
from itertools import product
from rich.pretty import pprint
from rich.console import Console

cmd = Console()

for inputs in product([False, True], repeat=2):
    output = not (inputs[0] and inputs[1])
    
    cmd.print(f"{inputs[0]}\t{inputs[1]}\t->\t{output}")

False   False   ->      True

False   True    ->      True

True    False   ->      True

True    True    ->      False

# QuBit


## 1. 

### 1.1 Not计算

我们可以用`|0>`表示0度状态的Qubit,`|1>`表示180度状态的Qubit。我们于是有`Not` 预算为

$$
Not \ket{0} = \ket{1}
$$

### 1.2 0~180度中间的ket

$$
(cos(\theta/2)\ket{0} + sin(\theta/2)\ket{1})
$$

or可以写成矩阵方式为:

$$
\begin{bmatrix} cos(\theta/2) \\ cos(\theta/2) \end{bmatrix}
$$

### 1.3 常用其他算子

$$
\ket{+} = \frac{1}{\sqrt{2}}(\ket{0} + \ket{1})
$$


$$
\ket{-} = \frac{1}{\sqrt{2}}(\ket{0} - \ket{1})
$$



## 2. 测量

`<*|`（读作bra）表示一个行向量，`|*>`（读作ket）表示一个列向量。`<*|`和`|*>`的乘积是一个数。这个数就是两个向量的内积。就是一个测量状态。比如`<0|1>`就是0度和180度的内积，结果是0，表示用0度的测量180度的概率为0。再比如，`<+|1>`就是90和180度的内积，结果是1/2，表示用+的测量180度的概率为1/2。


### 2.1 Hadamard算子

$$
H = \frac{1}{\sqrt{2}}\begin{bmatrix} 1 & 1 \\ 1 & -1 \end{bmatrix}
$$

一个计算的例子，用Hadamard算子对0度进行计算：

$$
H\ket{0} = \frac{1}{\sqrt{2}}\begin{bmatrix} 1 & 1 \\ 1 & -1 \end{bmatrix}\begin{bmatrix} 1 \\ 0 \end{bmatrix} = \frac{1}{\sqrt{2}}\begin{bmatrix} 1 \\ 1 \end{bmatrix} = \frac{1}{\sqrt{2}}(\ket{0} + \ket{1}) = \ket{+}
$$

Hadmard算子可以理解成45度的镜像。

正是世界中的QuBit是一个复数，所以我们可以用复数来表示QuBit的状态。这样，我们就需要一个Z轴来表示复数的实部和虚部。



# 3. QRNG

In [13]:
## RNG
def rng(seed: int) -> int:
    return (seed * 0x5DEECE66D + 0xB) & ((1 << 48) - 1)

In [17]:

def qrng():
    q = Qubit()
    H(q)
    return measure(q)

In [20]:
from abc import ABCMeta, abstractmethod
from contextlib import contextmanager


class Qubit(metaclass=ABCMeta):
    
    @abstractmethod
    def h(self):
        pass

    @abstractmethod
    def measure(self) -> bool:
        pass

    @abstractmethod
    def reset(self):
        pass


class QuantumDevice(metaclass=ABCMeta):
    @abstractmethod
    def allocate_qubit(self) -> Qubit:
        pass

    @abstractmethod
    def deallocate_qubit(self, qubit: Qubit):
        pass

    @contextmanager
    def using_qubit(self):
        qubit = self.allocate_qubit()
        try:
            yield qubit
        finally:
            qubit.reset()
            self.deallocate_qubit(qubit)
            

def qrng(device: QuantumDevice) -> bool:
    with device.using_qubit() as q:
        q.h()
        return q.measure()

In [21]:

H: np.ndarray[np.int32, np.dtype[np.int32]] = np.array([[1, 1], [1, -1]]) / np.sqrt(2)
KET_0: np.ndarray[np.int32, np.dtype[np.int32]] = np.array([[1], [0]])

class SimulatedQubit(Qubit):
    state: np.ndarray[np.int32, np.dtype[np.int32]]
    def __init__(self):
        self.reset()

    def h(self):
        self.state = H @ self.state

    def measure(self) -> bool:
        pr0 = np.abs(self.state[0, 0]) ** 2
        sample = np.random.random() <= pr0
        return bool(0 if sample else 1)

    def reset(self):
        self.state = KET_0.copy()

In [23]:
class SingleQubitSimulator(QuantumDevice):
    def __init__(self) -> None:
        self.qubit = SimulatedQubit()
    
    def allocate_qubit(self) -> Qubit:
        return self.qubit

    
    def deallocate_qubit(self, qubit: Qubit):
        qubit.reset()
        self.qubit = qubit


In [24]:
qsim = SingleQubitSimulator()
for idx_sample in range(10):
    random_sample = qrng(qsim)
    print(f"Our QRNG returned {random_sample}.")

Our QRNG returned False.
Our QRNG returned True.
Our QRNG returned False.
Our QRNG returned True.
Our QRNG returned True.
Our QRNG returned False.
Our QRNG returned True.
Our QRNG returned True.
Our QRNG returned True.
Our QRNG returned True.
